In [ ]:
# UNet 2.5D - Entrenamiento y Evaluación por Planos (Axial, Coronal, Sagital)

import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms
import torch.nn as nn
from copy import deepcopy
import pandas as pd

# ==========  Dispositivo ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

# ========== Modelo ==========
class DoubleConv2D(nn.Module):
    def __init__(self, in_ch, out_ch, dropout_prob=0.1):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=dropout_prob)
        )

    def forward(self, x): return self.conv(x)

class UNet2_5D(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, base_filters=64, dropout_prob=0.1):
        super().__init__()
        self.enc1 = DoubleConv2D(in_channels, base_filters, dropout_prob)
        self.enc2 = DoubleConv2D(base_filters, base_filters*2, dropout_prob)
        self.enc3 = DoubleConv2D(base_filters*2, base_filters*4, dropout_prob)
        self.pool = nn.MaxPool2d(2)

        self.bottom = DoubleConv2D(base_filters*4, base_filters*8, dropout_prob)

        self.up2 = nn.ConvTranspose2d(base_filters*8, base_filters*4, 2, stride=2)
        self.dec2 = DoubleConv2D(base_filters*8, base_filters*4, dropout_prob)

        self.up1 = nn.ConvTranspose2d(base_filters*4, base_filters*2, 2, stride=2)
        self.dec1 = DoubleConv2D(base_filters*4, base_filters*2, dropout_prob)

        self.up0 = nn.ConvTranspose2d(base_filters*2, base_filters, 2, stride=2)
        self.dec0 = DoubleConv2D(base_filters*2, base_filters, dropout_prob)

        self.out_conv = nn.Conv2d(base_filters, out_channels, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))
        b = self.bottom(self.pool(e3))
        d2 = self.up2(b)
        d2 = torch.cat([d2, e3], dim=1)
        d2 = self.dec2(d2)
        d1 = self.up1(d2)
        d1 = torch.cat([d1, e2], dim=1)
        d1 = self.dec1(d1)
        d0 = self.up0(d1)
        d0 = torch.cat([d0, e1], dim=1)
        d0 = self.dec0(d0)
        return torch.sigmoid(self.out_conv(d0))

# ========== Dataset ==========
class MyoSegDataset25D(Dataset):
    def __init__(self, image_root, mask_root, plane="axial"):
        self.samples = []
        self.weights = []

        for patient_id in sorted(os.listdir(image_root)):
            img_dir = os.path.join(image_root, patient_id, plane)
            mask_dir = os.path.join(mask_root, patient_id, plane)
            if not os.path.isdir(img_dir) or not os.path.isdir(mask_dir):
                continue

            filenames = sorted(os.listdir(img_dir))
            for i in range(1, len(filenames) - 1):
                triplet = [filenames[i - 1], filenames[i], filenames[i + 1]]
                img_paths = [os.path.join(img_dir, f) for f in triplet]
                mask_path = os.path.join(mask_dir, filenames[i])

                if all(os.path.exists(p) for p in img_paths) and os.path.exists(mask_path):
                    self.samples.append((img_paths, mask_path))
                    mask = np.array(Image.open(mask_path).convert("L")) > 0
                    pct_foreground = mask.sum() / mask.size
                    weight = 1.0 + 10.0 * pct_foreground
                    self.weights.append(weight)

    def __len__(self): return len(self.samples)

    def __getitem__(self, idx):
        img_paths, mask_path = self.samples[idx]
        imgs = [transforms.ToTensor()(Image.open(p).convert("L")) for p in img_paths]
        image_tensor = torch.cat(imgs, dim=0)
        mask_tensor = (transforms.ToTensor()(Image.open(mask_path).convert("L")) > 0).float()
        return image_tensor, mask_tensor

# ========== Métricas ==========
def dice_loss(pred, target, smooth=1.):
    intersection = (pred * target).sum()
    return 1 - ((2. * intersection + smooth) / (pred.sum() + target.sum() + smooth))

def dice_coef(pred, target, smooth=1.): return ((2. * (pred * target).sum() + smooth) / (pred.sum() + target.sum() + smooth)).item()
def iou_score(pred, target, smooth=1.): return (((pred * target).sum() + smooth) / (pred.sum() + target.sum() - (pred * target).sum() + smooth)).item()
def precision_score(pred, target, smooth=1.): return (((pred * target).sum() + smooth) / ((pred * target).sum() + (pred * (1 - target)).sum() + smooth)).item()
def recall_score(pred, target, smooth=1.): return (((pred * target).sum() + smooth) / ((pred * target).sum() + ((1 - pred) * target).sum() + smooth)).item()
def f1_score(pred, target, smooth=1.): return (2 * precision_score(pred, target, smooth) * recall_score(pred, target, smooth)) / (precision_score(pred, target, smooth) + recall_score(pred, target, smooth) + 1e-8)

# ========== Entrenamiento por plano ==========
DATA_DIR = "/kaggle/input/datsaset-total/New_UNet2.5D"
image_train = f"{DATA_DIR}/img_2D/img_train"
mask_train  = f"{DATA_DIR}/mask_2D/mask_train"
OUTPUT_DIR = "/kaggle/working"

PLANES = ["axial", "coronal", "sagital"]
EPOCHS = 30
BATCH_SIZE = 4
THRESHOLD_METRIC = 0.5
PATIENCE = 4
TOLERANCE = 0.01

for plane in PLANES:
    print(f"\n================== 🧬 ENTRENANDO PLANO: {plane.upper()} ==================\n")
    dataset = MyoSegDataset25D(image_train, mask_train, plane=plane)
    sampler = WeightedRandomSampler(dataset.weights, len(dataset.weights), replacement=True)
    train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=sampler)

    model = UNet2_5D().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

    best_loss = float("inf")
    best_model = None
    epochs_without_improvement = 0

    for epoch in range(EPOCHS):
        model.train()
        total_loss = 0

        for img, mask in train_loader:
            img, mask = img.to(device), mask.to(device)
            pred = model(img)
            bce = nn.BCELoss()(pred, mask)
            loss = 0.5 * dice_loss(pred, mask) + 0.5 * bce
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        print(f"📉 Epoch {epoch} | Loss: {avg_loss:.4f}")
        
        # Evaluar métricas en modo eval
        model.eval()
        with torch.no_grad():
            dice_vals, iou_vals, prec_vals, recall_vals, f1_vals = [], [], [], [], []
            for img, mask in train_loader:
                img, mask = img.to(device), mask.to(device)
                pred = model(img)
                pred_bin = (pred > THRESHOLD_METRIC).float()
                dice_vals.append(dice_coef(pred_bin, mask))
                iou_vals.append(iou_score(pred_bin, mask))
                prec_vals.append(precision_score(pred_bin, mask))
                recall_vals.append(recall_score(pred_bin, mask))
                f1_vals.append(f1_score(pred_bin, mask))
        
            avg_dice = np.mean(dice_vals)
            avg_iou = np.mean(iou_vals)
            avg_prec = np.mean(prec_vals)
            avg_recall = np.mean(recall_vals)
            avg_f1 = np.mean(f1_vals)
        
            print(f" Métricas | Dice: {avg_dice:.4f} | IoU: {avg_iou:.4f} | Prec: {avg_prec:.4f} | Recall: {avg_recall:.4f} | F1: {avg_f1:.4f}")


        model.eval()
        with torch.no_grad():
            idx = np.random.randint(len(dataset))
            sample_img, sample_mask = dataset[idx]
            input_tensor = sample_img.unsqueeze(0).to(device)
            output = model(input_tensor)
            pred_mask = (output > THRESHOLD_METRIC).float()

            fig, axs = plt.subplots(1, 3, figsize=(12, 4))
            axs[0].imshow(sample_img[1].cpu().numpy(), cmap="gray")
            axs[0].set_title(f"{plane} - Imagen central")
            axs[1].imshow(sample_mask.squeeze().cpu().numpy(), cmap="gray")
            axs[1].set_title("Máscara real")
            axs[2].imshow(pred_mask.squeeze().cpu().numpy(), cmap="gray")
            axs[2].set_title("Predicción")
            for ax in axs: ax.axis("off")
            plt.tight_layout()
            plt.show()

        if avg_loss < best_loss - TOLERANCE:
            best_loss = avg_loss
            best_model = deepcopy(model.state_dict())
            epochs_without_improvement = 0
            print(" Mejor modelo actualizado.")
        else:
            epochs_without_improvement += 1
            print(f"⏳ Sin mejora ({epochs_without_improvement}/{PATIENCE})")
            if epochs_without_improvement >= PATIENCE:
                print(" Early stopping activado.")
                break

    if best_model:
        torch.save(best_model, os.path.join(OUTPUT_DIR, f"unet2_5d_{plane}.pth"))
        print(f" Modelo guardado: unet2_5d_{plane}.pth")
    else:
        print(f" No se guardó ningún modelo para {plane.upper()} (no hubo mejora)")

# ========== Evaluación y comparativa ==========
print("\n EVALUACIÓN DE LOS TRES PLANOS\n")
results = []

for plane in PLANES:
    model_path = os.path.join(OUTPUT_DIR, f"unet2_5d_{plane}.pth")
    print(f"\n Evaluando modelo para plano: {plane.upper()}")
    if not os.path.exists(model_path):
        print(f" Modelo no encontrado para plano {plane.upper()} — saltando evaluación.")
        continue

    dataset = MyoSegDataset25D(image_root=image_train, mask_root=mask_train, plane=plane)
    loader = DataLoader(dataset, batch_size=1, shuffle=False)

    model = UNet2_5D().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    dice_list, iou_list, prec_list, recall_list, f1_list = [], [], [], [], []

    with torch.no_grad():
        for img, mask in loader:
            img, mask = img.to(device), mask.to(device)
            output = model(img)
            pred_bin = (output > THRESHOLD_METRIC).float()
            dice_list.append(dice_coef(pred_bin, mask))
            iou_list.append(iou_score(pred_bin, mask))
            prec_list.append(precision_score(pred_bin, mask))
            recall_list.append(recall_score(pred_bin, mask))
            f1_list.append(f1_score(pred_bin, mask))

    avg_dice = np.mean(dice_list)
    avg_iou = np.mean(iou_list)
    avg_prec = np.mean(prec_list)
    avg_recall = np.mean(recall_list)
    avg_f1 = np.mean(f1_list)

    results.append({
        "Plano": plane,
        "Dice": avg_dice,
        "IoU": avg_iou,
        "Precisión": avg_prec,
        "Recall": avg_recall,
        "F1": avg_f1
    })

    print(f" Dice: {avg_dice:.4f} | IoU: {avg_iou:.4f} | Precisión: {avg_prec:.4f} | Recall: {avg_recall:.4f} | F1: {avg_f1:.4f}")

    idx = np.random.randint(len(dataset))
    sample_img, sample_mask = dataset[idx]
    input_tensor = sample_img.unsqueeze(0).to(device)
    output = model(input_tensor)
    pred_mask = (output > THRESHOLD_METRIC).float()

    fig, axs = plt.subplots(1, 3, figsize=(12, 4))
    axs[0].imshow(sample_img[1].cpu().numpy(), cmap="gray")
    axs[0].set_title(f"{plane} - Imagen central")
    axs[1].imshow(sample_mask.squeeze().cpu().numpy(), cmap="gray")
    axs[1].set_title("Máscara real")
    axs[2].imshow(pred_mask.squeeze().cpu().numpy(), cmap="gray")
    axs[2].set_title("Predicción")
    for ax in axs: ax.axis("off")
    plt.tight_layout()
    plt.show()

if results:
    df = pd.DataFrame(results)
    print("\n Comparativa de métricas por plano:\n")
    print(df.set_index("Plano").round(4))
    df.to_csv(os.path.join(OUTPUT_DIR, "metricas_comparativas_planes.csv"), index=False)
    print(f"\n Resultados guardados en: {os.path.join(OUTPUT_DIR, 'metricas_comparativas_planes.csv')}")
else:
    print("\n No se ha evaluado ningún modelo. Asegúrate de haber completado el entrenamiento antes.")


In [ ]:
#INFERENCIA SOBRE EL TRAIN:
import os
import torch
import numpy as np
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# ========= Dataset ==========
class MyoSegDataset25D(Dataset):
    def __init__(self, image_root, mask_root, plane="axial"):
        self.samples = []
        self.patients = []

        for patient_id in sorted(os.listdir(image_root)):
            img_dir = os.path.join(image_root, patient_id, plane)
            mask_dir = os.path.join(mask_root, patient_id, plane)
            if not os.path.isdir(img_dir) or not os.path.isdir(mask_dir):
                continue

            filenames = sorted(os.listdir(img_dir))
            for i in range(1, len(filenames) - 1):
                triplet = [filenames[i - 1], filenames[i], filenames[i + 1]]
                img_paths = [os.path.join(img_dir, f) for f in triplet]
                mask_path = os.path.join(mask_dir, filenames[i])

                if all(os.path.exists(p) for p in img_paths) and os.path.exists(mask_path):
                    self.samples.append((img_paths, mask_path))
                    self.patients.append(patient_id)

    def __len__(self): return len(self.samples)

    def __getitem__(self, idx):
        img_paths, mask_path = self.samples[idx]
        imgs = [transforms.ToTensor()(Image.open(p).convert("L")) for p in img_paths]
        image_tensor = torch.cat(imgs, dim=0)
        filename = os.path.basename(mask_path)
        patient = self.patients[idx]
        return image_tensor, filename, patient


# ========= Modelo UNet 2.5D ==========
class DoubleConv2D(torch.nn.Module):
    def __init__(self, in_ch, out_ch, dropout_prob=0.1):
        super().__init__()
        self.conv = torch.nn.Sequential(
            torch.nn.Conv2d(in_ch, out_ch, 3, padding=1),
            torch.nn.BatchNorm2d(out_ch),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(out_ch, out_ch, 3, padding=1),
            torch.nn.BatchNorm2d(out_ch),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout2d(p=dropout_prob)
        )

    def forward(self, x): return self.conv(x)

class UNet2_5D(torch.nn.Module):
    def __init__(self, in_channels=3, out_channels=1, base_filters=64, dropout_prob=0.1):
        super().__init__()
        self.enc1 = DoubleConv2D(in_channels, base_filters, dropout_prob)
        self.enc2 = DoubleConv2D(base_filters, base_filters*2, dropout_prob)
        self.enc3 = DoubleConv2D(base_filters*2, base_filters*4, dropout_prob)
        self.pool = torch.nn.MaxPool2d(2)
        self.bottom = DoubleConv2D(base_filters*4, base_filters*8, dropout_prob)
        self.up2 = torch.nn.ConvTranspose2d(base_filters*8, base_filters*4, 2, stride=2)
        self.dec2 = DoubleConv2D(base_filters*8, base_filters*4, dropout_prob)
        self.up1 = torch.nn.ConvTranspose2d(base_filters*4, base_filters*2, 2, stride=2)
        self.dec1 = DoubleConv2D(base_filters*4, base_filters*2, dropout_prob)
        self.up0 = torch.nn.ConvTranspose2d(base_filters*2, base_filters, 2, stride=2)
        self.dec0 = DoubleConv2D(base_filters*2, base_filters, dropout_prob)
        self.out_conv = torch.nn.Conv2d(base_filters, out_channels, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))
        b = self.bottom(self.pool(e3))
        d2 = self.dec2(torch.cat([self.up2(b), e3], dim=1))
        d1 = self.dec1(torch.cat([self.up1(d2), e2], dim=1))
        d0 = self.dec0(torch.cat([self.up0(d1), e1], dim=1))
        return torch.sigmoid(self.out_conv(d0))


# ========= Inferencia por plano ==========
def infer_and_save(plane, model_path, image_root, mask_root, output_dir, threshold=0.5):
    print(f"\n🔍 Inferencia del plano: {plane.upper()}")

    # Cargar dataset
    dataset = MyoSegDataset25D(image_root, mask_root, plane=plane)
    loader = DataLoader(dataset, batch_size=1, shuffle=False)

    # Cargar modelo
    model = UNet2_5D().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    for img, fname, patient in tqdm(loader):
        img = img.to(device)
        with torch.no_grad():
            pred = model(img)
            pred_mask = (pred > threshold).float().squeeze().cpu().numpy() * 255

        save_path = os.path.join(output_dir, plane, patient[0])
        os.makedirs(save_path, exist_ok=True)
        Image.fromarray(pred_mask.astype(np.uint8)).save(os.path.join(save_path, fname[0]))

    print(f" Predicciones del plano {plane.upper()} guardadas en {os.path.join(output_dir, plane)}")


# ========= Configuración ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

DATASET_DIR = "/kaggle/input/datsaset-total/New_UNet2.5D"
PLANOS_DIR = "/kaggle/input/planos"
OUT_DIR = "/kaggle/working/predicciones_por_plano"

image_root = os.path.join(DATASET_DIR, "img_2D", "img_train")
mask_root = os.path.join(DATASET_DIR, "mask_2D", "mask_train")

planes = ["axial", "coronal", "sagital"]
for plane in planes:
    model_path = os.path.join(PLANOS_DIR, f"unet2_5d_{plane}.pth")
    infer_and_save(plane, model_path, image_root, mask_root, OUT_DIR)



In [ ]:
#INFERENCIA SOBRE EL TEST

import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# ========== 💻 Dispositivo ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

# ========== 🛠️ Modelo (misma UNet2_5D) ==========
class DoubleConv2D(torch.nn.Module):
    def __init__(self, in_ch, out_ch, dropout_prob=0.1):
        super().__init__()
        self.conv = torch.nn.Sequential(
            torch.nn.Conv2d(in_ch, out_ch, 3, padding=1),
            torch.nn.BatchNorm2d(out_ch),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(out_ch, out_ch, 3, padding=1),
            torch.nn.BatchNorm2d(out_ch),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout2d(p=dropout_prob)
        )

    def forward(self, x): return self.conv(x)

class UNet2_5D(torch.nn.Module):
    def __init__(self, in_channels=3, out_channels=1, base_filters=64, dropout_prob=0.1):
        super().__init__()
        self.enc1 = DoubleConv2D(in_channels, base_filters, dropout_prob)
        self.enc2 = DoubleConv2D(base_filters, base_filters*2, dropout_prob)
        self.enc3 = DoubleConv2D(base_filters*2, base_filters*4, dropout_prob)
        self.pool = torch.nn.MaxPool2d(2)
        self.bottom = DoubleConv2D(base_filters*4, base_filters*8, dropout_prob)
        self.up2 = torch.nn.ConvTranspose2d(base_filters*8, base_filters*4, 2, stride=2)
        self.dec2 = DoubleConv2D(base_filters*8, base_filters*4, dropout_prob)
        self.up1 = torch.nn.ConvTranspose2d(base_filters*4, base_filters*2, 2, stride=2)
        self.dec1 = DoubleConv2D(base_filters*4, base_filters*2, dropout_prob)
        self.up0 = torch.nn.ConvTranspose2d(base_filters*2, base_filters, 2, stride=2)
        self.dec0 = DoubleConv2D(base_filters*2, base_filters, dropout_prob)
        self.out_conv = torch.nn.Conv2d(base_filters, out_channels, 1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool(e1))
        e3 = self.enc3(self.pool(e2))
        b = self.bottom(self.pool(e3))
        d2 = self.up2(b)
        d2 = torch.cat([d2, e3], dim=1)
        d2 = self.dec2(d2)
        d1 = self.up1(d2)
        d1 = torch.cat([d1, e2], dim=1)
        d1 = self.dec1(d1)
        d0 = self.up0(d1)
        d0 = torch.cat([d0, e1], dim=1)
        d0 = self.dec0(d0)
        return torch.sigmoid(self.out_conv(d0))

# ========== 📂 Dataset solo para imágenes ==========
class MyoSegTestDataset25D(Dataset):
    def __init__(self, image_root, plane="axial"):
        self.samples = []
        for patient_id in sorted(os.listdir(image_root)):
            img_dir = os.path.join(image_root, patient_id, plane)
            if not os.path.isdir(img_dir): continue
            filenames = sorted(os.listdir(img_dir))
            for i in range(1, len(filenames) - 1):
                triplet = [filenames[i - 1], filenames[i], filenames[i + 1]]
                img_paths = [os.path.join(img_dir, f) for f in triplet]
                if all(os.path.exists(p) for p in img_paths):
                    self.samples.append(img_paths)

    def __len__(self): return len(self.samples)

    def __getitem__(self, idx):
        img_paths = self.samples[idx]
        imgs = [transforms.ToTensor()(Image.open(p).convert("L")) for p in img_paths]
        image_tensor = torch.cat(imgs, dim=0)
        return image_tensor

# ========== 🚀 Inferencia ==========
DATA_DIR = "/kaggle/input/datsaset-total/New_UNet2.5D"
WEIGHTS_DIR = "/kaggle/input/planos"
image_test = f"{DATA_DIR}/img_2D/img_test"
THRESHOLD_METRIC = 0.5
PLANES = ["axial", "coronal", "sagital"]

from tqdm import tqdm  # Para barra de progreso si quieres

mask_test = f"{DATA_DIR}/mask_2D/mask_test"  # Ruta a las máscaras reales de test

print("\n================== 🧪 INFERENCIA Y MÉTRICAS SOBRE TEST ==================")
for plane in PLANES:
    print(f"\n🔍 Evaluando plano: {plane.upper()}")
    model_path = os.path.join(WEIGHTS_DIR, f"unet2_5d_{plane}.pth")
    if not os.path.exists(model_path):
        print(f"⚠️ No se encontró el modelo para {plane}")
        continue

    dataset = MyoSegTestDataset25DWithMasks(image_root=image_test, mask_root=mask_test, plane=plane)
    if len(dataset) == 0:
        print("⚠️ No hay datos válidos para este plano.")
        continue

    loader = DataLoader(dataset, batch_size=1, shuffle=False)
    model = UNet2_5D().to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    dice_list, iou_list, prec_list, recall_list, f1_list = [], [], [], [], []

    with torch.no_grad():
        for img, mask in tqdm(loader):
            img, mask = img.to(device), mask.to(device)
            output = model(img)
            pred_bin = (output > THRESHOLD_METRIC).float()

            dice_list.append(dice_coef(pred_bin, mask))
            iou_list.append(iou_score(pred_bin, mask))
            prec_list.append(precision_score(pred_bin, mask))
            recall_list.append(recall_score(pred_bin, mask))
            f1_list.append(f1_score(pred_bin, mask))

    avg_dice = np.mean(dice_list)
    avg_iou = np.mean(iou_list)
    avg_prec = np.mean(prec_list)
    avg_recall = np.mean(recall_list)
    avg_f1 = np.mean(f1_list)

    print(f"✅ Resultados para {plane.upper()} — Dice: {avg_dice:.4f} | IoU: {avg_iou:.4f} | Precisión: {avg_prec:.4f} | Recall: {avg_recall:.4f} | F1: {avg_f1:.4f}")



In [ ]:
import shutil

shutil.make_archive("/kaggle/working/predicciones_sin_post", 'zip', "/kaggle/working/predicciones_test")


In [ ]:
#Visualización de las predicciones del test

import os
import random
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

IMG_DIR = "/kaggle/input/datsaset-total/New_UNet2.5D/img_2D/img_test"
PRED_DIR = "/kaggle/working/predicciones_test"
PLANOS = ["axial", "coronal", "sagital"]
NUM_EJEMPLOS = 10

# Recoger ejemplos válidos
ejemplos = []
for plano in PLANOS:
    plano_dir = os.path.join(PRED_DIR, plano)
    for paciente in os.listdir(plano_dir):
        pred_path = os.path.join(plano_dir, paciente)
        img_path = os.path.join(IMG_DIR, paciente, plano)
        if not os.path.isdir(pred_path) or not os.path.isdir(img_path):
            continue
        for fname in os.listdir(pred_path):
            img_name = fname
            orig_path = os.path.join(img_path, img_name)
            pred_path_img = os.path.join(pred_path, fname)
            if os.path.exists(orig_path):
                ejemplos.append((plano, paciente, img_name, orig_path, pred_path_img))

# Seleccionar aleatoriamente
ejemplos_mostrar = random.sample(ejemplos, min(NUM_EJEMPLOS, len(ejemplos)))

# Mostrar
for plano, paciente, fname, orig_path, pred_path in ejemplos_mostrar:
    img = np.array(Image.open(orig_path).convert("L"))
    pred = np.array(Image.open(pred_path).convert("L")) > 0

    # Normalizar imagen a [0, 1] y expandir a RGB
    img_rgb = np.stack([img] * 3, axis=-1) / 255.0

    # Crear overlay con canal rojo donde hay predicción
    overlay = img_rgb.copy()
    overlay[pred, 0] = 1.0   # rojo
    overlay[pred, 1] = 0.0   # quitar verde
    overlay[pred, 2] = 0.0   # quitar azul

    # Opcional: hacer mezcla alfa para transparencia
    alpha = 0.4
    overlay = (1 - alpha) * img_rgb + alpha * overlay

    fig, axs = plt.subplots(1, 3, figsize=(12, 4))

    axs[0].imshow(img, cmap="gray")
    axs[0].set_title("Imagen original")

    axs[1].imshow(pred, cmap="gray")
    axs[1].set_title("Máscara predicha")

    axs[2].imshow(overlay)
    axs[2].set_title("Overlay")

    for ax in axs:
        ax.axis("off")
    plt.suptitle(f"{plano.upper()} | Paciente: {paciente} | Archivo: {fname}", fontsize=10)
    plt.tight_layout()
    plt.show()


In [ ]:
#Post-procesado de las predicciones del conjunto de test

import os
from PIL import Image
import numpy as np
from tqdm import tqdm
from scipy.ndimage import binary_fill_holes, binary_closing, binary_erosion, binary_dilation
from skimage.morphology import remove_small_objects, disk
from skimage.filters import gaussian
from skimage.io import imsave
from skimage.segmentation import random_walker
import shutil

# Rutas
PRED_DIR = "predicciones_test"
IMG_DIR = "/kaggle/input/datsaset-total/New_UNet2.5D/img_2D/img_test"  # ⚠️ Asegúrate de tener las imágenes originales aquí (mismo nombre y estructura)
OUT_DIR = "predicciones_post_random_walker"
PLANOS = ["axial", "coronal", "sagital"]

def aplicar_random_walker(imagen_gray, mascara_binaria, fondo_dilatado=False):
    """
    Expande la máscara usando Random Walker guiado por la imagen original.
    """
    # Semillas: 1 donde hay máscara, 2 donde se asume fondo
    seeds = np.zeros_like(mascara_binaria, dtype=np.uint8)
    seeds[mascara_binaria > 0] = 1

    if fondo_dilatado:
        fondo = binary_dilation(mascara_binaria == 0, structure=disk(5))
        seeds[fondo & (seeds == 0)] = 2
    else:
        seeds[mascara_binaria == 0] = 2

    # Normalizamos imagen entre 0 y 1
    imagen_norm = (imagen_gray - imagen_gray.min()) / (imagen_gray.max() - imagen_gray.min() + 1e-8)

    try:
        labels = random_walker(imagen_norm, seeds, beta=250, mode='bf')
        resultado = (labels == 1).astype(np.uint8)
    except:
        # Si falla (por ejemplo imagen vacía), devolvemos la máscara original
        resultado = mascara_binaria.astype(np.uint8)

    return resultado

def postprocesar_mascara(mask_binaria, min_area=400, suavizar=True):
    """
    Limpieza morfológica clásica de la máscara binaria.
    """
    mask = mask_binaria.astype(bool)

    mask = remove_small_objects(mask, min_size=min_area)
    mask = binary_fill_holes(mask)
    mask = binary_closing(mask, structure=disk(8))

    if suavizar:
        mask = binary_erosion(mask, structure=disk(4))
        mask = gaussian(mask.astype(float), sigma=2) > 0.5

    return mask.astype(np.uint8)

# Procesamiento general
for plano in PLANOS:
    in_pred_dir = os.path.join(PRED_DIR, plano)
    out_dir = os.path.join(OUT_DIR, plano)
    os.makedirs(out_dir, exist_ok=True)

    print(f"\n🧠 Procesando plano: {plano.upper()}")

    for paciente in tqdm(os.listdir(in_pred_dir), desc=f"{plano.upper()}"):
        pred_paciente_dir = os.path.join(in_pred_dir, paciente)
        img_paciente_dir = os.path.join(IMG_DIR, paciente, plano)  # ✅ solo se añade plano aquí
        out_paciente_dir = os.path.join(out_dir, paciente)
        os.makedirs(out_paciente_dir, exist_ok=True)

        for fname in os.listdir(pred_paciente_dir):
            pred_path = os.path.join(pred_paciente_dir, fname)
            img_path = os.path.join(img_paciente_dir, fname)  # ✅ construida correctamente
            out_path = os.path.join(out_paciente_dir, fname)

            if not os.path.exists(img_path):
                print(f"⚠️ Imagen no encontrada, se omite: {img_path}")
                continue

            pred = np.array(Image.open(pred_path).convert("L")) > 0
            imagen = np.array(Image.open(img_path).convert("L"))

            pred_proc = postprocesar_mascara(pred, min_area=400)
            pred_final = aplicar_random_walker(imagen, pred_proc, fondo_dilatado=True)

            imsave(out_path, (pred_final * 255).astype(np.uint8))


In [ ]:
#Visualización y comparativa entre las predicciones del test con y sin post-procesado

import os
import random
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

# 📁 Directorios
IMG_DIR = "/kaggle/input/datsaset-total/New_UNet2.5D/img_2D/img_test"
PRED_DIR = "/kaggle/working/predicciones_test"
POST_DIR = "/kaggle/working/predicciones_post_random_walker" 
PLANOS = ["axial", "coronal", "sagital"]
NUM_EJEMPLOS = 10

# 🎨 Función overlay
def crear_overlay(img, mask, color=[1.0, 0.0, 0.0], alpha=0.4):
    img_rgb = np.stack([img]*3, axis=-1) / 255.0
    overlay = img_rgb.copy()
    overlay[mask] = color
    return (1 - alpha) * img_rgb + alpha * overlay

# 🔍 Buscar ejemplos válidos
ejemplos = []
for plano in PLANOS:
    plano_pred_dir = os.path.join(PRED_DIR, plano)
    plano_post_dir = os.path.join(POST_DIR, plano)

    for paciente in os.listdir(plano_pred_dir):
        pred_path = os.path.join(plano_pred_dir, paciente)
        post_path = os.path.join(plano_post_dir, paciente)
        img_path = os.path.join(IMG_DIR, paciente, plano)

        if not (os.path.isdir(pred_path) and os.path.isdir(post_path) and os.path.isdir(img_path)):
            continue

        for fname in os.listdir(pred_path):
            orig_img = os.path.join(img_path, fname)
            pred_img = os.path.join(pred_path, fname)
            post_img = os.path.join(post_path, fname)
            if os.path.exists(orig_img) and os.path.exists(post_img):
                ejemplos.append((plano, paciente, fname, orig_img, pred_img, post_img))

# 📢 Comprobación
if not ejemplos:
    print("⚠️ No se encontraron ejemplos válidos para visualizar.")
else:
    ejemplos_mostrar = random.sample(ejemplos, min(NUM_EJEMPLOS, len(ejemplos)))
    print(f"✅ Visualizando {len(ejemplos_mostrar)} ejemplos...")

    # 👁️ Mostrar
    for plano, paciente, fname, orig_path, pred_path, post_path in ejemplos_mostrar:
        img = np.array(Image.open(orig_path).convert("L"))
        pred = np.array(Image.open(pred_path).convert("L")) > 0
        post = np.array(Image.open(post_path).convert("L")) > 0

        overlay_pred = crear_overlay(img, pred)
        overlay_post = crear_overlay(img, post)

        fig, axs = plt.subplots(1, 4, figsize=(16, 4))

        axs[0].imshow(img, cmap="gray")
        axs[0].set_title("Imagen original")

        axs[1].imshow(overlay_pred)
        axs[1].set_title("Overlay predicción")

        axs[2].imshow(post, cmap="gray")
        axs[2].set_title("Postprocesada")

        axs[3].imshow(overlay_post)
        axs[3].set_title("Overlay postprocesada")

        for ax in axs:
            ax.axis("off")

        plt.suptitle(f"{plano.upper()} | Paciente: {paciente} | Archivo: {fname}", fontsize=10)
        plt.tight_layout()
        plt.show()


In [ ]:
# Comprimir en ZIP
shutil.make_archive("predicciones_post_random_walker", 'zip', OUT_DIR)
print("\n✅ Archivo ZIP generado: predicciones_post_random_walker.zip")